In [1]:
import pandas as pd
import datetime as dt

pd.options.display.max_columns=1000
pd.options.display.max_rows = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%run configs.ipynb

In [2]:
# parameter cell
file_name = 'oct_21.csv'

In [3]:
# Parameters
file_name = "june_21.csv"


# RRC Prod

## Ingest

In [4]:
production = pd.read_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', header = 5)

In [5]:
production.head()

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
0,NaN,Gas,05,164066.0,DUBOIS (BARNETT),MILES,1,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
1,NaN,Gas,7B,211352.0,"GRANBURY, N.E. (ATOKA)",LANGDON,2,NaN,5422.0,4,0,NaN,NaN,4,0,NaN,NaN
2,NaN,Gas,7B,212409.0,"GRANBURY, N.E. (ATOKA)","JOHN J. MILES, JR. GAS UNIT",4,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
3,NaN,Gas,7B,228695.0,"GRANBURY, N.E. (ATOKA)",LUCY UNIT,2,NaN,5699.0,12,0,NaN,NaN,12,82,82,3.0
4,NaN,Gas,7B,228926.0,"GRANBURY, N.E. (ATOKA)","MARSHALL, SAM A",2,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN


## Transform RRC

### Filter to District

In [6]:
#converting data field to string to handle varying schema interpreations 
#(certain months field will be interpreted as float and string depending sample rows pandas uses

production = production.loc[production['District'].astype(str).isin(['8.0', '08', '8'])]

In [7]:
production

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
7,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,276,46,137,1.0,185,0,NaN,NaN
8,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,78,0,39,1.0,39,0,NaN,NaN
9,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,333,109,175,1.0,267,"1,191","1,191",2.0
10,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,638","7,873","7,498",0.0,"2,013","18,248","18,248",2.0
11,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,323,"1,224","1,183",0.0,364,"3,680","3,680",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,NaN,Gas,08,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,10,24,30,1.0,4,880,880,2.0
466,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
467,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,6,24,26,1.0,4,87,87,2.0
468,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,7,24,27,1.0,4,390,390,2.0


In [8]:
production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 7 to 469
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Multiple Reports            80 non-null     object 
 1   Lease Type                  407 non-null    object 
 2   District                    460 non-null    object 
 3   RRC Identifier              460 non-null    float64
 4   Field Name                  460 non-null    object 
 5   Lease Name                  460 non-null    object 
 6   Gas Well ID                 232 non-null    object 
 7   Lease Total                 13 non-null     object 
 8   Commingle Permit No.        273 non-null    float64
 9   On Hand Beginning of Month  460 non-null    object 
 10  Production                  460 non-null    object 
 11  Volume                      198 non-null    object 
 12  Code                        198 non-null    float64
 13  On Hand End of Month        460 non

### Clean Oil Production Columns and Convert to Int

In [9]:
production['Volume'] = production.loc[:,'Volume'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))
production['Volume.1'] = production.loc[:,'Volume.1'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))

### Rename Volume to rrc_oil_volume

In [10]:
production.rename(columns = {'Volume': 'rrc_oil_volume'}, inplace = True)
production.rename(columns = {'Volume.1': 'rrc_gas_volume'}, inplace = True)

In [11]:
production['rrc_oil_volume'].sum()
production['rrc_gas_volume'].sum()

357811

1291553

### Normalize Field and Reservoir Names with Text Cleaning

In [12]:
production["Normalized_Field_Name"] = production['Field Name'].dropna()\
                                                            .apply(lambda x: str(x).split('(')[0]\
                                                                                    .split(',')[0]\
                                                                                    .strip())

In [13]:
production["Normalized_Reservoir_Name"] = production['Field Name'].dropna()\
                                        .apply(lambda x: str(x).split('(')[-1]\
                                                               .strip().replace(')',''))

### Get Date from Production File

In [14]:
def get_date_from_csv(filepath, col_number, row_number):
    str_date = pd.read_csv(filepath, usecols = [col_number]).iloc[row_number].values[0]
    return str_date.strip()

In [15]:
get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1)

'Jun 2021'

### Add Production Date to Production Data Frame

In [16]:
production.insert(0, 'Date', get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1))

In [17]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name
7,Jun 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,276,46,137,1.0,185,0,0,NaN,ARMER,TUBB
8,Jun 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,78,0,39,1.0,39,0,0,NaN,ARMER,6350
9,Jun 2021,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,333,109,175,1.0,267,"1,191",1191,2.0,ARMER,6350
10,Jun 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,638","7,873",7498,0.0,"2,013","18,248",18248,2.0,ARMER,6350
11,Jun 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,323,"1,224",1183,0.0,364,"3,680",3680,2.0,ARMER,6350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,Jun 2021,NaN,Gas,08,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,10,24,30,1.0,4,880,880,2.0,WADDELL,GRAYBURG
466,Jun 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
467,Jun 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,6,24,26,1.0,4,87,87,2.0,WADDELL,GRAYBURG
468,Jun 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,7,24,27,1.0,4,390,390,2.0,WADDELL,GRAYBURG


### Add columns for net production to Trust

In [18]:
production['Net Oil Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50
production['Net Oil Sales Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50 * 0.75

In [19]:
production['Net Gas Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50
production['Net Gas Sales Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50 * 0.75

In [20]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name,Net Oil Volume to Trust (RRC),Net Oil Sales Volume to Trust (RRC),Net Gas Volume to Trust (RRC),Net Gas Sales Volume to Trust (RRC)
7,Jun 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,276,46,137,1.0,185,0,0,NaN,ARMER,TUBB,68.5,51.375,0.0,0.000
8,Jun 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,78,0,39,1.0,39,0,0,NaN,ARMER,6350,19.5,14.625,0.0,0.000
9,Jun 2021,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,333,109,175,1.0,267,"1,191",1191,2.0,ARMER,6350,87.5,65.625,595.5,446.625
10,Jun 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,638","7,873",7498,0.0,"2,013","18,248",18248,2.0,ARMER,6350,3749.0,2811.750,9124.0,6843.000
11,Jun 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,323,"1,224",1183,0.0,364,"3,680",3680,2.0,ARMER,6350,591.5,443.625,1840.0,1380.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,Jun 2021,NaN,Gas,08,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,10,24,30,1.0,4,880,880,2.0,WADDELL,GRAYBURG,15.0,11.250,440.0,330.000
466,Jun 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
467,Jun 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,6,24,26,1.0,4,87,87,2.0,WADDELL,GRAYBURG,13.0,9.750,43.5,32.625
468,Jun 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,7,24,27,1.0,4,390,390,2.0,WADDELL,GRAYBURG,13.5,10.125,195.0,146.250


### Write to File

In [21]:
production.to_csv(f'{processed_data_folder}/rrc_prod/{file_name}', index = False)

## Aggregated EDA

In [22]:
production.groupby(['Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'})\
                                            .sort_values(by = 'rrc_oil_volume', ascending= False)\
                                            .head(15)

,rrc_oil_volume
Normalized_Field_Name,
SAND HILLS,196799
MONAHANS,70916
ARMER,36131
DUNE,17102
UNIVERSITY WADDELL,10126
MARSTON RANCH,8519
WADDELL,6729
RUNNING W,6413
CORDONA LAKE,2424


In [23]:
production.groupby(['Normalized_Reservoir_Name', 'Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'}).sort_values(by = 'rrc_oil_volume', ascending= False).sort_index()

rrc_oil_volume
Normalized_Reservoir_Name Normalized_Field_Name                
6350                      ARMER                           35994
7900                      EDWARDS -04-                        0
CLEAR FORK                LEA                               313
                          MONAHANS                            0
                          SAND HILLS                      44724
CLEAR FORK 4070           SAND HILLS                        246
CLEAR FORK, LOWER         MCKEE                               0
CLEAR FORK, MID.          MCKEE                               0
CLEARFORK                 MARSTON RANCH                    8519
                          MONAHANS                        65842
                          RM                                  0
COLBY                     KEYSTONE                            0
CONSOLIDATED              KEYSTONE                            0
CORDONA LAKE              CORDONA LAKE                      328
COWDEN, NORTH             COWDEN                              0
DEV.                      CORDONA LAKE                     1707
DEVONIAN                  CORDONA LAKE                      172
                          UNIVERSITY WADDELL               9466
                          WADDELL                            59
DUNE                      DUNE                            17047
ELLENBURGER               KERMIT                              0
                          SAND HILLS                        928
                          UNIVERSITY WADDELL                  0
GRAYBURG                  GOLDSMITH                           0
                          WADDELL                           386
HOLT                      RUNNING W                          77
                          SAND HILLS                          0
JUDKINS                   SAND HILLS                      11671
KERMIT                    KERMIT                              0
MCKEE                     BLOCK 27                            0
                          KERMIT                              0
                          RUNNING W                           0
MCKNIGHT                  SAND HILLS                      32564
MONAHANS, NORTH           MONAHANS                         5074
ORDOVICIAN                SAND HILLS                          0
PENN UPPER                BLOCK 31                          349
PENN.                     G. P. M.                            0
                          UNIVERSITY WADDELL                 24
QUEEN                     GOLDSMITH                           0
SAN ANDRES                C-BAR                               0
                          DELL                              165
                          KEYSTONE                            0
                          LEA                                53
                          SHAFTER LAKE                      538
SAN ANDRES CONS.          CRAWAR                              0
SAN ANDRES, CON.          GOLDSMITH                           0
SAN ANGELO                LEA                              1234
                          SAND HILLS                       1965
SAN ANGELO, UPPER         SAND HILLS                      21201
TUBB                      -Y-                                 0
                          ARMER                             137
                          BLOCK B-21                          0
                          RUNNING W                        1141
                          SAND HILLS                      39885
TUBB 4500                 CORDONA LAKE                      217
WADDELL                   RUNNING W                        4441
                          WADDELL                          6284
WADDELL SD.               UNIVERSITY WADDELL                636
WICHITA-ALBANY            RUNNING W                         754
                          SAND HILLS                      21478
WICHITA-ALBANY, GAS       SAND HILLS                       1472
WOJCIK-MCELROY   